In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

np.random.seed(42)
n_samples, n_features = 500, 7
base_corr = 0.9
cov = np.full((n_features, n_features), base_corr)
np.fill_diagonal(cov, 1.0)
mean = np.zeros(n_features)
X = np.random.multivariate_normal(mean, cov, size=n_samples)
true_weights = np.array([2.5, -1.8, 1.2, 0.8, 0.5, 1.5, -0.7])
bias = 3.0
noise = np.random.normal(0, 1.5, size=n_samples)
y = X.dot(true_weights) + bias + noise

scaler = StandardScaler()
X = scaler.fit_transform(X)

def ridge_gradient_descent(X, y, lr, reg_lambda, n_iters=1000):
    m, n = X.shape
    X_b = np.c_[np.ones((m, 1)), X]
    theta = np.zeros((n + 1, 1))
    y = y.reshape(-1, 1)
    for _ in range(n_iters):
        gradients = (2/m) * X_b.T.dot(X_b.dot(theta) - y)
        gradients[1:] += 2 * reg_lambda * theta[1:]
        theta -= lr * gradients
        if np.isnan(theta).any() or np.isinf(theta).any():
            return None
    return theta

learning_rates = [0.0001, 0.001, 0.01, 0.05]
lambdas = [1e-5, 1e-3, 0.01, 0.1, 1, 10]

best_score, best_params = -np.inf, None
for lr in learning_rates:
    for l in lambdas:
        theta = ridge_gradient_descent(X, y, lr, l)
        if theta is None:
            continue
        y_pred = np.c_[np.ones((len(X), 1)), X].dot(theta)
        if np.isnan(y_pred).any():
            continue
        r2 = r2_score(y, y_pred)
        if r2 > best_score:
            best_score, best_params = r2, (lr, l)

print("Best Parameters: Learning Rate =", best_params[0], ", Lambda =", best_params[1])
print("Best R² Score:", round(best_score, 4))


Best Parameters: Learning Rate = 0.05 , Lambda = 1e-05
Best R² Score: 0.8708


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv("Hitters.csv")
df = df.dropna()
df = pd.get_dummies(df, drop_first=True)

X = df.drop('Salary', axis=1)
y = df['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

models = {
    'Linear': LinearRegression(),
    'Ridge': Ridge(alpha=0.5748, max_iter=10000),
    'Lasso': Lasso(alpha=0.5748, max_iter=10000)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(name, "Regression:")
    print("MSE:", mean_squared_error(y_test, y_pred))
    print("R2 Score:", r2_score(y_test, y_pred))


Linear Regression:
MSE: 128284.34549672354
R2 Score: 0.290745185579814
Ridge Regression:
MSE: 126603.90264424692
R2 Score: 0.30003596988293446
Lasso Regression:
MSE: 126678.11604014733
R2 Score: 0.29962566098567167


In [12]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

data = fetch_california_housing()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

ridge = RidgeCV(alphas=[0.1, 1, 10], cv=5)
lasso = LassoCV(alphas=[0.1, 1, 10], cv=5, max_iter=10000)

ridge.fit(X_train, y_train)
lasso.fit(X_train, y_train)

print("Ridge Best Alpha:", ridge.alpha_)
print("Ridge R2:", r2_score(y_test, ridge.predict(X_test)))
print("Lasso Best Alpha:", lasso.alpha_)
print("Lasso R2:", r2_score(y_test, lasso.predict(X_test)))


Ridge Best Alpha: 0.1
Ridge R2: 0.5757905180002312
Lasso Best Alpha: 0.1
Lasso R2: 0.4813611325029077


In [9]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load data
iris = load_iris()
X, y = iris.data, iris.target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train Logistic Regression (one-vs-rest)
model = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=1000)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9555555555555556

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.85      0.92        13
           2       0.87      1.00      0.93        13

    accuracy                           0.96        45
   macro avg       0.96      0.95      0.95        45
weighted avg       0.96      0.96      0.96        45



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
